# 使用自定义的联邦算法训练模型

[上一节](8.%20%E8%87%AA%E5%AE%9A%E4%B9%89%E8%81%94%E9%82%A6%E5%AD%A6%E4%B9%A0%E7%AE%97%E6%B3%95.ipynb)定义了一个极简的 FedAvg 算法实现，现在用这个算法调度器来训练[第二节](2.%20%E5%9C%A8%20AlphaMed%20%E5%B9%B3%E5%8F%B0%E4%B8%8A%E8%BF%90%E8%A1%8C%E6%99%AE%E9%80%9A%E6%A8%A1%E5%9E%8B.ipynb)里定义的 `ConvNet` 模型，数据集依然使用 MNIST 数据集。

## 提供继承 `SimpleFedAvgScheduler` 调度器的实现

`SimpleFedAvgScheduler` 已经提供了完整的算法流程控制，所以这部分不需要再关心。但是其中还有五个与具体任务相关的接口，要提供实现。

In [ ]:
from abc import ABCMeta, abstractmethod
from torch.nn import Module
from torch.utils.tensorboard import SummaryWriter

@abstractmethod
def before_check_in(self, is_aggregator: bool):
    """完成集合前初始化本地资源。"""

@abstractmethod
def before_training(self, is_aggregator: bool):
    """训练开始前的初始化工作。"""

@property
@abstractmethod
def model(self) -> Module:
    """获取训练使用的模型对象。"""

@abstractmethod
def train_an_epoch(self):
    """完成一个 epoch 训练的逻辑。"""

@abstractmethod
def test(self, writer: SummaryWriter):
    """测试的逻辑。"""

`before_check_in` 接口是在集合流程之前执行的，可以在此接口中加载训练需要的数据集。如果数据集加载失败，也可以提前告知任务管理器任务失败，没必要继续运行浪费时间。

In [ ]:
import os
from torch.utils.data import DataLoader
import torchvision

def before_check_in(self, is_aggregator: bool):
    """完成集合前初始化本地资源。"""
    if is_aggregator:
        self.train_loader = DataLoader(
            torchvision.datasets.MNIST(
                'data',
                train=True,
                download=True,
                transform=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
            ),
            batch_size=64,
            shuffle=True
        )
    else:
        self.test_loader = DataLoader(
            torchvision.datasets.MNIST(
                'data',
                train=False,
                download=True,
                transform=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
            ),
            batch_size=64,
            shuffle=False
        )

`before_training` 是在完成集合之后，开始训练之前执行的。可以在此时准备一下优化器。这个操作对聚合方、参与方都是一样的，所以可以忽略 `is_aggregator` 参数。

In [ ]:
from torch.optim import SGD

def before_training(self, is_aggregator: bool):
    """训练开始前的初始化工作。"""
    self.optimizer = SGD(params=self.model.parameters(), lr=0.01, momentum=0.9)

模型使用 `ConvNet`，注意生成模型后要将实例保护起来，避免每次返回一个新的实例，导致之前的参数更新丢失。

In [ ]:
from res.cnn_net import ConvNet

@property
def model(self) -> Module:
    """获取训练使用的模型对象。"""
    if not hasattr(self, '_model'):
        self._model = ConvNet()
    return self._model

在极简 FedAvg 算法中，训练只发生在参与方节点上，因此不需要考虑聚合方的情况。

In [ ]:
import torch.nn.functional as F

def train_an_epoch(self):
    """完成一个 epoch 训练的逻辑。"""
    self.model.train()
    for data, labels in self.train_loader:
        self.optimizer.zero_grad()
        output = self.model(data)
        loss = F.nll_loss(output, labels)
        loss.backward()
        self.optimizer.step()

在极简 FedAvg 算法中，测试只发生在聚合方节点上，因此不需要考虑参与方的情况。

In [ ]:
import torch

def test(self, writer: SummaryWriter):
    """测试的逻辑。"""
    self.model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, labels in self.test_loader:
            output: torch.Tensor = self.model(data)
            test_loss += F.nll_loss(output, labels, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(labels.view_as(pred)).sum().item()

    test_loss /= len(self.test_loader.dataset)
    correct_rate = 100. * correct / len(self.test_loader.dataset)
    writer.add_scalar('test_results/average_loss', test_loss, 1)
    writer.add_scalar('test_results/correct_rate', correct_rate, 1)

整理一下得到完整的调度器实现。

In [ ]:
import torch
import torch.nn.functional as F
import torchvision
from torch.nn import Module
from torch.optim import SGD
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from cnn_net import ConvNet
from simple_fed_avg.scheduler import SimpleFedAvgScheduler


class MyScheduler(SimpleFedAvgScheduler):

    def __init__(self, rounds: int) -> None:
        super().__init__(rounds)

    def before_check_in(self, is_aggregator: bool):
        """完成集合前初始化本地资源。"""
        if is_aggregator:
            self.test_loader = DataLoader(
                torchvision.datasets.MNIST(
                    'data',
                    train=False,
                    download=True,
                    transform=torchvision.transforms.Compose([
                        torchvision.transforms.ToTensor(),
                        torchvision.transforms.Normalize((0.1307,), (0.3081,))
                    ])
                ),
                batch_size=64,
                shuffle=False
            )
        else:
            self.train_loader = DataLoader(
                torchvision.datasets.MNIST(
                    'data',
                    train=True,
                    download=True,
                    transform=torchvision.transforms.Compose([
                        torchvision.transforms.ToTensor(),
                        torchvision.transforms.Normalize((0.1307,), (0.3081,))
                    ])
                ),
                batch_size=64,
                shuffle=True
            )

    def before_training(self, is_aggregator: bool):
        """训练开始前的初始化工作。"""
        self.optimizer = SGD(params=self.model.parameters(), lr=0.01, momentum=0.9)

    @property
    def model(self) -> Module:
        """获取训练使用的模型对象。"""
        if not hasattr(self, '_model'):
            self._model = ConvNet()
        return self._model

    def train_an_epoch(self):
        """完成一个 epoch 训练的逻辑。"""
        self.model.train()
        for data, labels in self.train_loader:
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = F.nll_loss(output, labels)
            loss.backward()
            self.optimizer.step()

    def run_test(self, writer: SummaryWriter):
        """测试的逻辑。"""
        self.model.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, labels in self.test_loader:
                output: torch.Tensor = self.model(data)
                test_loss += F.nll_loss(output, labels, reduction='sum').item()
                pred = output.max(1, keepdim=True)[1]
                correct += pred.eq(labels.view_as(pred)).sum().item()

        test_loss /= len(self.test_loader.dataset)
        correct_rate = 100. * correct / len(self.test_loader.dataset)
        writer.add_scalar('test_results/average_loss', test_loss, 1)
        writer.add_scalar('test_results/correct_rate', correct_rate, 1)

实际执行之前，最好先在模拟环境调试一下，以提前暴露 bug，方便修改。

## 在模拟环境中调试自定义算法

在模拟环境中，由于缺乏真正的任务管理器，所以启动模拟环境时需要提供一些预设参数。可以参考[横向联邦学习环境简介](3.%20%E6%A8%AA%E5%90%91%E8%81%94%E9%82%A6%E5%AD%A6%E4%B9%A0%E7%8E%AF%E5%A2%83%E7%AE%80%E4%BB%8B.ipynb)中的相关介绍。

In [ ]:
import logging
from alphafed import logger, mock_context

# 关闭 DEBUG 日志，如果需要更详细的调试日志，可以将这一行注释掉
logger.setLevel(logging.INFO)

# 聚合方的模拟启动脚本
scheduler = MyScheduler(rounds=5)

aggregator_id = '4d41ca74-078c-494c-bf7f-324534460d10'  # 指定一个假想 ID
col_id_1 = 'e71690e4-347c-4fda-be6f-c8d49040c692'  # 指定一个假想 ID
col_id_2 = 'fab1898f-ed8a-47de-ba17-62fcd8916189'  # 指定一个假想 ID
task_id = 'a701073d-e2bb-401b-a1f0-bf2a8e58dd2a'  # 指定一个假想 ID

with mock_context(id=aggregator_id, nodes=[aggregator_id, col_id_1, col_id_2]):
    scheduler._run(id=aggregator_id, task_id=task_id, is_initiator=True)


# 参与方的模拟启动脚本，需要复制到单独的 Notebook 脚本文件中执行
# scheduler 实例和聚合方的一模一样
scheduler = MyScheduler(rounds=5)

aggregator_id = '4d41ca74-078c-494c-bf7f-324534460d10'  # 与聚合方一致
col_id_1 = 'e71690e4-347c-4fda-be6f-c8d49040c692'  # 与聚合方一致
col_id_2 = 'fab1898f-ed8a-47de-ba17-62fcd8916189'  # 与聚合方一致
task_id = 'a701073d-e2bb-401b-a1f0-bf2a8e58dd2a'  # 与聚合方一致
with mock_context(id=col_id_1, nodes=[aggregator_id, col_id_1, col_id_2]):
    scheduler._run(id=col_id_1, task_id=task_id)


# 另一个参与方的模拟启动脚本，需要复制到单独的 Notebook 脚本文件中执行
# scheduler 实例和聚合方的一模一样
scheduler = MyScheduler(rounds=3)

aggregator_id = '4d41ca74-078c-494c-bf7f-324534460d10'  # 与聚合方一致
col_id_1 = 'e71690e4-347c-4fda-be6f-c8d49040c692'  # 与聚合方一致
col_id_2 = 'fab1898f-ed8a-47de-ba17-62fcd8916189'  # 与聚合方一致
task_id = 'a701073d-e2bb-401b-a1f0-bf2a8e58dd2a'  # 与聚合方一致
with mock_context(id=col_id_2, nodes=[aggregator_id, col_id_1, col_id_2]):
    scheduler._run(id=col_id_2, task_id=task_id)

整理好的[聚合方脚本](res/9_aggregator.ipynb)、[参与方-1 脚本](res/9_collaborator_1.ipynb)、[参与方-2 脚本](res/9_collaborator_2.ipynb)均可以直接运行。

在模拟环境调试运行成功之后，对聚合方的启动脚本稍作修改，就可以启动正式任务了。执行横向联邦学习任务需要登录 [AlphaMed Playground 客户端](https://alphamed.ssplabs.com/)，[这里](../fed_avg/README.ipynb)有创建横向联邦学习任务的详细说明，请按照说明中的步骤运行示例程序。

In [ ]:
# 聚合方的模拟启动脚本
scheduler = MyScheduler(rounds=5)  # 受本地资源限制，运行速度可能会比较慢

# 这些模拟调试的代码不需要了
# aggregator_id = '4d41ca74-078c-494c-bf7f-324534460d10'  # 指定一个假想 ID
# col_id_1 = 'e71690e4-347c-4fda-be6f-c8d49040c692'  # 指定一个假想 ID
# col_id_2 = 'fab1898f-ed8a-47de-ba17-62fcd8916189'  # 指定一个假想 ID
# task_id = 'a701073d-e2bb-401b-a1f0-bf2a8e58dd2a'  # 指定一个假想 ID
# with mock_context(id=aggregator_id, nodes=[aggregator_id, col_id_1, col_id_2]):
#     scheduler._run(id=aggregator_id, task_id=task_id, is_initiator=True)

scheduler.submit(task_id='YOUR_TASK_ID')  # 正式启动学习任务